## Imports and utils

In [39]:
import networkx as nx
import csv
import time
import pandas as pd
from cdlib import algorithms, readwrite, evaluation
from cdlib.benchmark import LFR, SBM
import infomap

In [4]:
def datafileToGraph(fileName):
    emailRaw = pd.read_csv(fileName, header=None)
    emailRaw = emailRaw[0].str.split(n=2, expand=True)
    emailRaw.columns = ['Source', 'Target']
    #print(emailRaw)
    emailNetwork = nx.from_pandas_edgelist(emailRaw, source='Source', target='Target', edge_attr=None)
    #nx.draw(emailNetwork)  
    return emailNetwork
emailNet = datafileToGraph('emailNet.txt')
print(emailNet)

Graph with 1005 nodes and 16706 edges


In [2]:
def averageDegree(networkx):
    degrees = [val for (node, val) in networkx.degree()]
    sum = 0
    for d in degrees:
        sum += d
    return sum/len(degrees)

## Email network properties 

In [38]:
print("Degree distribution:", nx.degree_histogram(emailNet))
print("Average degree:", averageDegree(emailNet))
print("Clustering coefficient:", nx.average_clustering(emailNet))

for C in (emailNet.subgraph(c) for c in nx.connected_components(emailNet)):
    print("Average Path Length:", nx.average_shortest_path_length(C))
    break
    
#centrality

Degree distribution: [0, 78, 46, 42, 33, 28, 21, 25, 20, 15, 14, 20, 15, 16, 14, 14, 12, 9, 12, 15, 18, 15, 16, 13, 16, 18, 14, 11, 12, 20, 19, 11, 10, 8, 9, 8, 11, 11, 9, 6, 7, 9, 8, 7, 7, 9, 9, 8, 7, 5, 6, 9, 8, 7, 7, 8, 5, 4, 3, 3, 4, 5, 5, 6, 5, 2, 1, 2, 3, 2, 4, 5, 4, 5, 6, 1, 3, 1, 1, 1, 3, 2, 1, 5, 6, 3, 4, 2, 1, 1, 0, 3, 3, 1, 0, 1, 1, 1, 2, 1, 0, 2, 0, 1, 0, 1, 3, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 2, 1, 2, 1, 0, 1, 1, 1, 0, 0, 1, 1, 2, 1, 0, 1, 1, 0, 1, 2, 2, 0, 2, 3, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## Community finding methods for the email network

In [6]:
start_time = time.time()
emailLouvain = algorithms.louvain(emailNet)
print("Execution time for Louvain in email net: %.6s seconds" % (time.time() - start_time))
start_time = time.time()
emailGN = algorithms.girvan_newman(emailNet, level=3)
print("Execution time for Girvan-Newman in email net: %.6s seconds" % (time.time() - start_time))
#start_time = time.time()
#emailKC = algorithms.kclique(emailNet, k=10)
#print("Execution time for KCliques in email net: %.6s seconds" % (time.time() - start_time))
emailInfomap = infomap.Infomap(silent=True, num_trials=10)
emailInfomap.read_file("emailNet.txt")
start_time = time.time()
emailInfomap.run()
print("Execution time for Infomap in email net: %.6s seconds" % (time.time() - start_time))

Execution time for Louvain in email net: 0.5029 seconds
Execution time for Girvan-Newman in email net: 133.70 seconds
Execution time for Infomap in email net: 0.3427 seconds


## Louvain properties

In [7]:
size = evaluation.size(emailNet, emailLouvain)
ad = evaluation.avg_distance(emailNet, emailLouvain)
aid = evaluation.average_internal_degree(emailNet, emailLouvain)
ae = evaluation.avg_embeddedness(emailNet, emailLouvain)
at = evaluation.avg_transitivity(emailNet, emailLouvain)
hd = evaluation.hub_dominance(emailNet, emailLouvain)
s = evaluation.significance(emailNet, emailLouvain)

print("Louvain Size:", size)
print("Louvain Average Path Length:",  ad)
print("Louvain Average Internal Degree:",  aid)
print("Louvain Average Embeddedness:",  ae)
print("Louvain Average Transitivity:",  at)
print("Louvain Hub Dominance:",  hd)
print("Louvain Significance:", s)

Louvain Size: FitnessResult(min=1, max=257, score=37.22222222222222, std=64.24105836804772)
Louvain Average Path Length: FitnessResult(min=0, max=2.3859131809338523, score=0.6372732939437186, std=0.9862965581428322)
Louvain Average Internal Degree: FitnessResult(min=2.0, max=25.21818181818182, score=7.197809938082192, std=8.238721261331312)
Louvain Average Embeddedness: FitnessResult(min=0.5467100007648499, max=1.0, score=0.9177967687574559, std=0.13313223291224136)
Louvain Average Transitivity: FitnessResult(min=0.0, max=0.7755090957031264, score=0.1607747947913536, std=0.2532535395442663)
Louvain Hub Dominance: FitnessResult(min=0.40625, max=0.7121212121212122, score=0.562919781067364, std=0.08929459213891795)
Louvain Significance: FitnessResult(min=None, max=None, score=61883.41182869238, std=None)


In [10]:
louvainDict = louvain.to_node_community_map()
#print(louvainDict)
#louvain.to_json()

## Girvan-Newman

In [8]:
size = evaluation.size(emailNet, emailGN)
ad = evaluation.avg_distance(emailNet, emailGN)
aid = evaluation.average_internal_degree(emailNet, emailGN)
ae = evaluation.avg_embeddedness(emailNet, emailGN)
at = evaluation.avg_transitivity(emailNet, emailGN)
hd = evaluation.hub_dominance(emailNet, emailGN)
s = evaluation.significance(emailNet, emailGN)

print("Girvan-Newman Size:", size)
print("Girvan-Newman Average Path Length:",  ad)
print("Girvan-Newman Average Internal Degree:",  aid)
print("Girvan-Newman Average Embeddedness:",  ae)
print("Girvan-Newman Average Transitivity:",  at)
print("Girvan-Newman Hub Dominance:",  hd)
print("Girvan-Newman Significance:", s)

Girvan-Newman Size: FitnessResult(min=1, max=978, score=43.69565217391305, std=199.1969605981256)
Girvan-Newman Average Path Length: FitnessResult(min=0, max=2.5665982212565908, score=0.1927506473010112, std=0.6330518282579853)
Girvan-Newman Average Internal Degree: FitnessResult(min=0.0, max=34.08997955010225, score=3.235796212323286, std=6.602873143511623)
Girvan-Newman Average Embeddedness: FitnessResult(min=0.0, max=1.0, score=0.9057542629926452, std=0.2815706562689681)
Girvan-Newman Average Transitivity: FitnessResult(min=0.0, max=0.4108588094169628, score=0.017863426496389688, std=0.08378689716311363)
Girvan-Newman Hub Dominance: FitnessResult(min=0.35516888433981575, max=1.2, score=0.7775844421699079, std=0.4224155578300921)
Girvan-Newman Significance: FitnessResult(min=None, max=None, score=78331.79378130106, std=None)


## Modularity

In [14]:
louvainGN = evaluation.newman_girvan_modularity(emailNet, emailLouvain)
louvainER = evaluation.erdos_renyi_modularity(emailNet, emailLouvain)
louvainZ = evaluation.z_modularity(emailNet, emailLouvain)

print("Louvain Girvan-Newman mod:", louvainGN)
print("Louvain Erdos-Renyi mod:", louvainER)
print("Louvain Z-mod:", louvainZ)

Louvain Girvan-Newman mod: FitnessResult(min=None, max=None, score=0.4338307368583568, std=None)
Louvain Erdos-Renyi mod: FitnessResult(min=None, max=None, score=0.4523182754359461, std=None)
Louvain Z-mod: FitnessResult(min=None, max=None, score=1.1687975670090407, std=None)


## External Evaluation

In [10]:
nmi = evaluation.normalized_mutual_information(emailLouvain, emailGN)
ami = evaluation.adjusted_mutual_information(emailLouvain, emailGN)
ari = evaluation.adjusted_rand_index(emailLouvain, emailGN)
f1 = evaluation.f1(emailLouvain, emailGN)
voi = evaluation.variation_of_information(emailLouvain, emailGN)

print("Normalized Mutual Information between Louvain and Girvan-Newman:", nmi)
print("Adjusted Mutual Information between Louvain and Girvan-Newman:", ami)
print("Adjusted Rand Index between Louvain and Girvan-Newman:", ari)
print("F1 measure between Louvain and Girvan-Newman:", f1)
print("Variation of information between Louvain and Girvan-Newman:", voi)

Normalized Mutual Information between Louvain and Girvan-Newman: MatchingResult(score=0.14504937959297817, std=None)
Adjusted Mutual Information between Louvain and Girvan-Newman: MatchingResult(score=0.10711820790370227, std=None)
Adjusted Rand Index between Louvain and Girvan-Newman: MatchingResult(score=0.01441831643609563, std=None)
F1 measure between Louvain and Girvan-Newman: MatchingResult(score=0.7681481481481481, std=0.3606274063355062)
Variation of information between Louvain and Girvan-Newman: MatchingResult(score=2.8252100650534304, std=None)


## Email Labels import 

In [12]:
def convertTextDatasetToCSV(txtFile, csvFile):
    txtFile = open(txtFile, 'r')
    
    txtLines = txtFile.readlines()
    csvLines = {}
    
    for line in txtLines:
        lineVec = line[:-1].split(' ')
        if lineVec[1] in csvLines.keys():
            csvLines[lineVec[1]].append(lineVec[0])
        else:
            csvLines[lineVec[1]] = [lineVec[0]]
            
    with open(csvFile, 'w',newline="") as csv_file:  
        writer = csv.writer(csv_file)
        for key, value in csvLines.items():
            writer.writerow(value)
        
convertTextDatasetToCSV('emailLabels.txt', 'emailLabels.csv')
emailLabels = readwrite.read_community_csv("emailLabels.csv", ",", str)

In [16]:
nmiLouvainLabels = emailLouvain.normalized_mutual_information(emailLabels)
nmiGirvanNewmanLabels = emailGN.normalized_mutual_information(emailLabels)

amiLouvainLabels = emailLouvain.adjusted_mutual_information(emailLabels)
amiGirvanNewmanLabels = emailGN.adjusted_mutual_information(emailLabels)

ariLouvainLabels = emailLouvain.adjusted_rand_index(emailLabels)
ariGirvanNewmanLabels = emailGN.adjusted_rand_index(emailLabels)

f1LouvainLabels = emailLouvain.f1(emailLabels)
f1GirvanNewmanLabels = emailGN.f1(emailLabels)

voiLouvainLabels = emailLouvain.variation_of_information(emailLabels)
voiGirvanNewmanLabels = emailGN.variation_of_information(emailLabels)

print("NMI for Louvain and labels:", nmiLouvainLabels)
print("NMI for Girvan-Newman and labels:", nmiGirvanNewmanLabels)
print()
print("AMI for Louvain and labels:", amiLouvainLabels)
print("AMI for Girvan-Newman and labels:", amiGirvanNewmanLabels)
print()
print("ARI for Louvain and labels:", ariLouvainLabels)
print("NMI for Girvan-Newman and labels:", ariGirvanNewmanLabels)
print()
print("F1 for Louvain and labels:", f1LouvainLabels)
print("F1 for Girvan-Newman and labels:", f1GirvanNewmanLabels)
print()
print("VOI for Louvain and labels:", voiLouvainLabels)
print("VOI for Girvan-Newman and labels:", voiGirvanNewmanLabels)

NMI for Louvain and labels: MatchingResult(score=0.5945484327084933, std=None)
NMI for Girvan-Newman and labels: MatchingResult(score=0.0449480988954229, std=None)

AMI for Louvain and labels: MatchingResult(score=0.5595133980763424, std=None)
AMI for Girvan-Newman and labels: MatchingResult(score=-0.0005067215292382119, std=None)

ARI for Louvain and labels: MatchingResult(score=0.3343323597723664, std=None)
NMI for Girvan-Newman and labels: MatchingResult(score=-0.0012481583268392242, std=None)

F1 for Louvain and labels: MatchingResult(score=0.21814814814814817, std=0.2837452827088648)
F1 for Girvan-Newman and labels: MatchingResult(score=0.05695652173913045, std=0.050599431394699676)

VOI for Louvain and labels: MatchingResult(score=3.1623719262379413, std=None)
VOI for Girvan-Newman and labels: MatchingResult(score=4.848380933791979, std=None)


## LFR 

In [49]:
n = 250
tau1 = 3
tau2 = 1.5
mu = 0.1
lfrGraph, comsLFR = LFR(n, tau1, tau2, mu, average_degree=5, min_community=20)
print(lfrGraph)
lfrLouvain = algorithms.louvain(lfrGraph)
lfrLeiden = algorithms.leiden(lfrGraph)
nmi1 = evaluation.normalized_mutual_information(lfrLouvain, comsLFR)
print(nmi1)
nmi2 = evaluation.normalized_mutual_information(lfrLeiden, comsLFR)
print(nmi2)

Graph with 250 nodes and 513 edges
MatchingResult(score=0.9680909762018572, std=None)
MatchingResult(score=1.0, std=None)


## SBM 

In [43]:
sizes = [75, 75, 300]
probs = [[0.25, 0.05, 0.02], [0.05, 0.35, 0.07], [0.02, 0.07, 0.40]]
sbmGraph, comsSBM = SBM(sizes, probs, seed=0)
print(sbmGraph)

Graph named 'stochastic_block_model' with 450 nodes and 22160 edges
